# Class Separation Transformation DR/feature-selection Algorithm Testing

In [2]:
#import useful libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
import sys
#import umap.umap_ as umap
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.decomposition import KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import KFold

In [2]:
#import data (beware, it takes around 10 minutes)
df = pd.read_csv('../clinical_TumorCompendium_v11_PolyA_2020-04-09.tsv', sep='\t', header=0)
df2 = pd.read_csv('../TumorCompendium_v11_PolyA_hugo_log2tpm_58581genes_2020-04-09.tsv', sep='\t', header=0)

MemoryError: Unable to allocate 5.56 GiB for an array with shape (12747, 58581) and data type float64

In [14]:
Y.shape

(8, 8)

In [17]:
Y[0]

array([ 533, 5643, 5345,  655,  145,  167,  264,  742])

In [15]:
#testing the CST_fit() method
Y = np.array([[533,5643,5345,655,145,167,264,742],[673,8743,45,6,15,6437,2664,7342],[5333,57643,57345,5,1745,67,4,72]
              ,[8765,3453,1235,879,985,123,7654,7422],[25233,15643,85345,5655,545,127,274,7522],[5332,56243,533845,6855,1845,1674,8264,7442]
              ,[5,53,55,6,15,17,24,42],[5383,7643,2345,7655,98245,1877,56664,1742], [6437,7347,2367,745,7684,1003,6743,6536]]) #9x8
#tester = np.where(Y,Y,Y.T) #to make sure it is symmetric and it is
ytester = np.array([0,0,0,0,1,1,1,1,1]) # 9x1
CST_fit(Y, ytester) #doesn't make a difference if you pass Y or tester

(array([ 2.50249299e+03, -2.09921390e+03,  2.89811522e+04, -1.81831906e+02,
         6.91050876e+04,  4.39032228e+05,  2.57149651e+01, -2.61687243e+04,
        -2.12199689e+03]),
 array([ 0.09308269, -0.35694523,  0.86241458, -0.1708543 , -0.23198583,
         0.18732832, -0.03836333, -0.02438336]))

In [24]:
#testing the CST_fit() method
Y = np.array([[533,5643,5345,655,145,167,264,742],[673,8743,45,6,15,6437,2664,7342],[5333,57643,57345,5,1745,67,4,72]
              ,[8765,3453,1235,879,985,123,7654,7422],[25233,15643,85345,5655,545,127,274,7522],[5332,56243,533845,6855,1845,1674,8264,7442]
              ,[5,53,55,6,15,17,24,42],[5383,7643,2345,7655,98245,1877,56664,1742], [6437,7347,2367,745,7684,1003,6743,6536]]) #9x8
#tester = np.where(Y,Y,Y.T) #to make sure it is symmetric and it is
ytester = np.array([0,0,0,0,1,1,1,1,1]) # 9x1
CST_fit(Y, ytester) #doesn't make a difference if you pass Y or tester

(array([ 2.50249299e+03, -2.09921390e+03,  2.89811522e+04, -1.81831906e+02,
         6.91050876e+04,  4.39032228e+05,  2.57149651e+01, -2.61687243e+04,
        -2.12199689e+03]),
 array([ 0.09308269, -0.35694523,  0.86241458, -0.1708543 , -0.23198583,
         0.18732832, -0.03836333, -0.02438336]))

In [ ]:
X_trans, f = CST_fit(X_train, y_train) #gets the transformed data and the associated vector minimizer f

In [10]:
def indicator_func(y, i, j):
    
    if(y[i] == y[j]):
        return 1
    else:
        return -1
    

In [14]:
#transform X into X_trans using CST algorithm
def CST_fit(X, y):
    
    #construct upper triangular A
    sampleNum = X.shape[0] #get the number of samples
    featureNum = X.shape[1] #get the number of features
    temp_A = np.zeros((featureNum, featureNum)) #A has dimensions nxn, devised from outer product of difference of two samples's feature values
    
    #loops over SxS where S:{1,m} (m = # of samples)
    for i in range(sampleNum):
        #print("Progess:", i)
        for j in range(i, sampleNum):
            
            #I think I need a condition that says as long as i != j 
            #(actually don't because it's adding 0's, eliminating saves flops though)
            if i != j:
                sample_difference = X[i] - X[j]
                temp_A += indicator_func(y,i,j)*(np.outer(sample_difference, sample_difference.T))
                #sample_dif_sqrd = sample_difference.reshape(-1,1) @ sample_difference.reshape(-1,1).T
                #temp_A[i][j] = indicator_func(y, i, j)*sample_dif_sqrd
                #temp_A += indicator_func(y, i, j)*sample_dif_sqrd
    
    #make matrix symmetric
    #A = np.where(temp_A, temp_A, temp_A.T)
    #print(temp_A) #prints the A matrix to make sure it is symmetric
    
    #calculate eigenvalues and eigenvectors of A
    eigvals, eigvecs = np.linalg.eig(temp_A)
    
    #get minimum eigenvector
    min_ind = np.argmin(eigvals)
    min_eigvec = eigvecs[:,min_ind] #in the algorithm, this is the vector f
    
    #transform the data
    X_trans = X @ min_eigvec 
    
    return X_trans, min_eigvec
    

In [12]:
def CST_transform(X, y, f):
    
    #transform the data
    X_trans = np.transpose(f) @ X
    
    return X_trans
    

### Test time for outer products of differing sizes

In [4]:
import time

In [162]:
#time how long it takes for one outer product in CST for this dataset
start = time.time()
np.outer(X_train[0], X_train[1])
end = time.time()
print(end - start)

8.30176329612732


In [19]:
test1 = np.random.rand(100)
test2 = np.random.rand(100)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.0


In [20]:
test1 = np.random.rand(1000)
test2 = np.random.rand(1000)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.016012907028198242


In [24]:
test1 = np.random.rand(2500)
test2 = np.random.rand(2500)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.04790639877319336


In [25]:
test1 = np.random.rand(4000)
test2 = np.random.rand(4000)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.10765194892883301


In [23]:
test1 = np.random.rand(5000)
test2 = np.random.rand(5000)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.15719389915466309


In [22]:
test1 = np.random.rand(8000)
test2 = np.random.rand(8000)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.40102601051330566


In [6]:
start = time.time()
for i in range(1300): #for 1300 samples we need 720,600 outer loop iterations
    test1 = np.random.rand(8000)
    test2 = np.random.rand(8000)
    np.outer(test1, test2)
end = time.time()
print(end - start)

255.6153745651245


In [21]:
test1 = np.random.rand(10000)
test2 = np.random.rand(10000)
start = time.time()
np.outer(test1, test2)
end = time.time()
print(end - start)

0.6265442371368408


In [ ]:
#cells below are testing the binary classifiers to make sure the shapes are correct (they weren't but now are)

In [110]:
df_samples = df[df['disease']=='acute myeloid leukemia']

samples = df_samples[df_samples['pedaya']=='No']['th_sampleid']
pos_samples = df2.loc[:,list(samples)]

samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
neg_samples = df2.loc[:,list(samples)]

pos_samples.shape

x = pd.concat([pos_samples.T, neg_samples.T]) #add together all postiive and negative labels 
x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

target_zeros = pd.DataFrame(np.zeros((len(neg_samples.T), 1))) #get number of target zeros
target_ones = pd.DataFrame(np.ones((len(pos_samples.T), 1))) #get number of target ones
target = pd.concat([target_ones, target_zeros]) #concatenate target ones and zeros together
target = target.reset_index(drop=True)

#split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)

In [112]:
print(pos_samples.shape)
print(X_train.shape)

(58581, 160)
(326, 58581)


In [141]:
df_samples = df[df['disease']=='lung squamous cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples11 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples11 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney clear cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples12 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples12 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='thyroid carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples13 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples13 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='head & neck squamous cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples14 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples14 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney papillary cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples15 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples15 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='bladder urothelial carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples16 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples16 = df2.loc[:,list(samples)]

neg_samples12.shape
#np.concatenate(neg_samples, neg_samples13)
temperneg = pd.concat([neg_samples11.T, neg_samples12.T, neg_samples13.T, neg_samples14.T, neg_samples15.T, neg_samples16.T])
temperpos = pd.concat([pos_samples11.T, pos_samples12.T, pos_samples13.T, pos_samples14.T, pos_samples15.T, pos_samples16.T])

In [150]:
x = pd.concat([temperneg, temperpos]) 
x.shape

(2762, 58581)

In [152]:
x = pd.concat([temperneg, temperpos]) #add together all postiive and negative labels 
x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

target_zeros = pd.DataFrame(np.zeros((len(temperneg), 1))) #get number of target zeros
target_ones = pd.DataFrame(np.ones((len(temperpos), 1))) #get number of target ones
target = pd.concat([target_ones, target_zeros]) #concatenate target ones and zeros together
target = target.reset_index(drop=True)

#split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)

In [153]:
X_train.shape

(1933, 58581)

In [154]:
x

array([[-0.14037443, -0.02683328, -0.28931516, ...,  0.        ,
        -0.3607229 , -0.24700628],
       [-0.14037443, -0.02683328, -0.28931516, ...,  0.        ,
        -0.33775742, -0.24700628],
       [-0.14037443, -0.02683328,  0.86913107, ...,  0.        ,
         0.28057589, -0.24700628],
       ...,
       [-0.14037443, -0.02683328, -0.28931516, ...,  0.        ,
        -1.41784984, -0.24700628],
       [-0.14037443, -0.02683328, -0.28931516, ...,  0.        ,
         0.73630094, -0.24700628],
       [-0.14037443, -0.02683328,  1.55898756, ...,  0.        ,
        -0.60884267, -0.24700628]])

In [138]:
print(np.count_nonzero(~np.isnan(x)))
print(np.count_nonzero(np.isnan(x)))

27357327
0


In [139]:
df_samples = df[df['disease']=='lung adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples27 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples27 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='stomach adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples28 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples28 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='prostate adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples29 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples29 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='colon adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples30 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples30 = df2.loc[:,list(samples)]

In [140]:
pos_samples6 = pd.concat([pos_samples27.T, pos_samples28.T, pos_samples29.T, pos_samples30.T])
neg_samples6 = pd.concat([neg_samples27.T, neg_samples28.T, neg_samples29.T, neg_samples30.T])

In [155]:
g1_samples = pd.concat([pos_samples27.T, pos_samples28.T, pos_samples29.T, pos_samples30.T])
g2_samples = pd.concat([neg_samples27.T, neg_samples28.T, neg_samples29.T, neg_samples30.T])

x = pd.concat([g1_samples, g2_samples]) #add together all postiive and negative labels 
x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

target_g1 = pd.DataFrame(np.zeros((len(g1_samples), 1))) #get number of target group 1
target_g2 = pd.DataFrame(np.ones((len(g2_samples), 1))) #get number of target group 2

target = pd.concat([target_g1, target_g2]) #concatenate target zeros ones twos etc together
target = target.reset_index(drop=True)

In [156]:
x

array([[-0.19286981, -0.02431083,  0.86393793, ...,  0.        ,
         1.06946242,  3.28324882],
       [ 5.44939562, -0.02431083,  0.15993541, ...,  0.        ,
         1.37012581,  0.105719  ],
       [-0.19286981, -0.02431083,  0.10680934, ...,  0.        ,
         2.72669997, -0.30247536],
       ...,
       [-0.19286981, -0.02431083, -0.35445028, ...,  0.        ,
         1.26255297, -0.30247536],
       [-0.19286981, -0.02431083, -0.35445028, ...,  0.        ,
         0.50939855, -0.30247536],
       [-0.19286981, -0.02431083, -0.35445028, ...,  0.        ,
         1.13081234,  1.09414353]])

In [157]:
print(np.count_nonzero(~np.isnan(x)))
print(np.count_nonzero(np.isnan(x)))

99177633
0
